# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# loading dotenv environment variables
%reload_ext dotenv # got error message that dotenv was already loaded, so I just reloaded it
%dotenv 




In [5]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
# see what I am working with
print(prices_dd.columns)
print(prices_dd._meta.index.name)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'Year'],
      dtype='object')
ticker


In [19]:
import os
from glob import glob

# load PRICE_DATA environment vairable
PRICE_DATA = os.environ["Price_Data"]

# final all the parquet files recursively under PRICE_DATE

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)


# read parquet files into a Dask DataFrame
prices_dd = dd.read_parquet(parquet_files).set_index("ticker")



For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:


# fixing earlier mistake, If ticker is currently the index, bring it back as a column
dd_feat = prices_dd.reset_index()

# Sort so shifts happen in the correct order within each ticker
dd_feat = dd_feat.sort_values(["ticker", "Date"])

# Add lagged values, daily returns, and high–low price range per ticker
dd_feat = prices_dd.reset_index().sort_values(["ticker", "Date"]).assign(
    Close_lag_1=lambda df: df.groupby("ticker")["Close"].shift(1, meta=("Close_lag_1", "float64")),
    Adj_Close_lag_1=lambda df: df.groupby("ticker")["Adj Close"].shift(1, meta=("Adj_Close_lag_1", "float64")),
)

dd_feat = dd_feat.assign(
    returns=lambda df: (df["Close"] / df["Close_lag_1"]) - 1,
    hi_lo_range=lambda df: df["High"] - df["Low"],
)






+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [27]:
dd_feat.head()

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
0,AAMC,2012-12-13,15.0,15.0,15.0,15.0,15.0,100.0,AAMC.csv,2012,NaN,NaN,NaN,0.0
1,AAMC,2012-12-14,19.0,30.0,19.0,30.0,30.0,144600.0,AAMC.csv,2012,12.74,11.923630,1.354788,11.0
2,AAMC,2012-12-17,31.5,65.0,31.5,65.0,65.0,68600.0,AAMC.csv,2012,12.74,11.923630,4.102041,33.5
3,AAMC,2012-12-18,65.0,89.0,65.0,80.0,80.0,43600.0,AAMC.csv,2012,9.25,8.657267,7.648649,24.0
4,AAMC,2012-12-19,80.0,84.0,78.0,84.0,84.0,24000.0,AAMC.csv,2012,10.19,9.537031,7.243376,6.0


In [ ]:
# trouble shooting an issue with my index
dd_feat.columns

Index(['ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'source', 'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns',
       'hi_lo_range'],
      dtype='object')

In [ ]:
# trouble shooting an issue with my index
type(dd_feat)

dask.dataframe.dask_expr._collection.DataFrame

In [ ]:
# trouble shooting an issue with my index
print(dd_feat._meta.columns)
print(list(dd_feat._meta.columns))

Index(['ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'source', 'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns',
       'hi_lo_range'],
      dtype='object')
['ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source', 'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns', 'hi_lo_range']


In [ ]:
# trouble shooting an issue with my index
cols = list(dd_feat.columns)
dupes = sorted({c for c in cols if cols.count(c) > 1})
print("Duplicate columns:", dupes)



Duplicate columns: []


In [ ]:
# trouble shooting an issue with my index
cols = list(dd_feat.columns)
dupes = sorted({c for c in cols if cols.count(c) > 1})
print("Duplicate columns:", dupes)

Duplicate columns: []


In [ ]:
# This took a long time, I messed something up in the beginning, looking forward to your comments
# Convert the Dask DataFrame to a pandas DataFrame.
# Ticker is stored as the index, so reset_index() makes it a regular column
# before loading everything into memory with compute().
pdf_base = prices_dd.reset_index().compute()


# Sort by ticker and date to ensure time-based operations
# (lags and rolling windows) are calculated in the correct order.
pdf_base = pdf_base.sort_values(["ticker", "Date"])

# Create lagged features for Close and Adjusted Close prices per ticker.
pdf_base["Close_lag_1"] = pdf_base.groupby("ticker")["Close"].shift(1)
pdf_base["Adj_Close_lag_1"] = pdf_base.groupby("ticker")["Adj Close"].shift(1)

# Compute daily returns based on the Close price.
pdf_base["returns"] = (pdf_base["Close"] / pdf_base["Close_lag_1"]) - 1

# Compute the daily high–low price range.
pdf_base["hi_lo_range"] = pdf_base["High"] - pdf_base["Low"]

# Add a 10-day moving average of returns for each ticker.
pdf_base["returns_ma_10"] = (
    pdf_base.groupby("ticker")["returns"]
            .rolling(10)
            .mean()
            .reset_index(level=0, drop=True)
)

pdf_feat = pdf_base

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It was not strictly necessary to convert the data to pandas to compute the moving average return, since Dask does support rolling window operations. However, the task explicitly required converting the Dask DataFrame to a pandas DataFrame, and once the data was in memory, pandas offered a simpler and more predictable way to compute the rolling mean. While performing this calculation in Dask would be more appropriate for very large datasets that do not fit into memory, rolling and grouping operations in Dask require careful partitioning to avoid compute-time issues. Given that the dataset was manageable in size and the final output needed to be in pandas, computing the moving average in pandas was the most reliable and practical choice in this case.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.